In [1]:
import sys
sys.path.append('..')

### Set OpenAI key 

In [46]:
import os
import configparser

config = configparser.ConfigParser()
config.read('../../.secrets.ini')
OPENAI_API_KEY = config['OPENAI']['OPENAI_API_KEY']
YOUTUBE_KEY = config['YOUTUBE']['YOUTUBE_API_KEY']
NAVER_CLIENT_ID = config['NAVER']['NAVER_CLIENT_ID']
NAVER_CLIENT_SECRET = config['NAVER']['NAVER_CLIENT_SECRET']
GOOGLE_SEARCH_KEY = config['GOOGLE']['GOOGLE_API_KEY']
CSE_ID = config['GOOGLE']['CSE_ID']
SERPAPI_API_KEY = config['SERPAPI']['SERPAPI_API_KEY']


os.environ.update({'OPENAI_API_KEY': OPENAI_API_KEY})
os.environ.update({'YOUTUBE_KEY': YOUTUBE_KEY})
os.environ.update({'NAVER_CLIENT_ID': NAVER_CLIENT_ID})
os.environ.update({'NAVER_CLIENT_SECRET': NAVER_CLIENT_SECRET})
os.environ.update({'GOOGLE_SEARCH_KEY': GOOGLE_SEARCH_KEY})
os.environ.update({'CSE_ID': CSE_ID})
os.environ.update({'SERPAPI_API_KEY': SERPAPI_API_KEY})

### Get tools

In [32]:
# %load_ext autoreload
# %autoreload 2
%reload_ext autoreload

In [ ]:
from importlib import reload
import tools
reload(tools)

In [34]:
from tools import DatabaseTool, SearchTool, SearchByURLTool, TimeTool
from models.llm.chain import SummaryChunkChain

In [16]:
summary_chunk_template = '''
I want you to answer the question from the given documentation. 
There may be a lot of information in a given document that is not relevant to your question, but you should try to find as much relevant information as possible.
Be as diligent as possible in returning your answer based on your knowledge and documentation.
If there is no answer in the document, return "No useful information".

Document: {document}
Question: {question}
Answer:
'''

In [35]:
summary_chunk_chain = SummaryChunkChain(summary_chunk_template=summary_chunk_template, input_variables=["question", "document"], verbose=True)
search_by_url_tool = SearchByURLTool()

In [56]:
from langchain.agents import Tool
from langchain.utilities import SerpAPIWrapper

# You can create the tool to pass to an agent
search = SerpAPIWrapper(params={
        "engine": "google",
        "google_domain": "google.com",
        "gl": "kr",
        "hl": "ko",
        })
search_tool = Tool(
    name="search_tool",
    description="A tool to search on internet.",
    func=search.run,
)

In [36]:
database_tool = DatabaseTool(summary_chunk_chain=summary_chunk_chain)
search_tool = SearchTool(summary_chunk_chain=summary_chunk_chain, search_by_url_tool=search_by_url_tool)
time_tool = TimeTool()

In [57]:
tools = [database_tool, search_tool, time_tool]

In [9]:
from embedchain.embedchain import EmbedChain
from embedchain.config import AppConfig
embed_chain = EmbedChain(config=AppConfig())

In [10]:
from database import DataBase

In [ ]:
database = DataBase.load(database_path='./user/test_2/database.json', embed_chain=embed_chain)

### Set Agent

In [12]:
from models.llm.qna_agent import QnAAgent

In [58]:
qna_agent = QnAAgent(tools=tools, verbose=True, qna_prompt_path='../../openai_skt/models/templates/qna_prompt_template.txt', model='gpt-4')

In [61]:
answer = qna_agent.run(database=database, query="오늘 제주도 날씨가 어때요?")



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. Use a variety of tools to answer the questions. You have access to the following tools:

database: A tool to get data with from database. The input consists of a 'query' and a 'question', where 'query' is the search query and 'question' is the information you want to get. For example, {'query': 'bitcoin price history', 'question': 'what was the price of bitcoin in 2022?'} would be the input if you want to know the price of bitcoin in 2022. Input must contain both a query and a question.
search_tool: A tool to search on internet.
Time Tool: Tool to get current time. There is no input


Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [database, search_tool, Time Tool]
Action Input: the input to the action
Observation: the res

In [42]:
from langchain.agents import load_tools
tools = load_tools(["serpapi"])

ValidationError: 1 validation error for SerpAPIWrapper
__root__
  Did not find serpapi_api_key, please add an environment variable `SERPAPI_API_KEY` which contains it, or pass  `serpapi_api_key` as a named parameter. (type=value_error)

In [15]:
search_tool.search(query="비트코인 전망", category='google')

{'google_search': [{'title': "'9월 하락 징크스' 비트코인, 이더리움과 리플 전망은?",
   'description': '비트코인(Bitcoin, BTC)은 8월 이후 12.3% 하락한 2만 5700달러, ... 연중 고점을 회복한 순간 단기 보유자 투자 심리 회복이 기대된다고 전망했다.',
   'data_type': 'web_page',
   'data_path': 'https://www.blockmedia.co.kr/archives/364701'},
  {'title': '비트코인 3700만 원대 하락, 비트코인 2028년 3억8천만 원 ...',
   'description': '[비즈니스포스트]비트코인 시세가 3700만 원대로 내려왔다.비트코인 가격이 2028년에는 30만 달러(약 3억8500만 원)에 달할 것이라는 전망이 나온다.',
   'data_type': 'web_page',
   'data_path': 'https://www.businesspost.co.kr/BP?command=article_view&num=322643'},
  {'title': "“비트코인에 최고의 호재”...10% 뛴 데다 '불장' 전망까지 나온 근거",
   'description': '20일 가상화폐 거래소 업비트에 따르면 이날 비트코인은 3500만원 선에 거래됐다. 지난 6월 15일 3250만원대까지 떨어졌던 것에 비해 9.4% 가량 상승한 ...',
   'data_type': 'web_page',
   'data_path': 'https://stock.mk.co.kr/news/view/153470'}]}